# Create features from the SCHUFA-Auskunft

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_path = "/media/veracrypt1/SPON_dataset_tables/"
plot_path = "plots/"

In [3]:
#df_ds = pd.read_csv(data_path + "SPON_dataset_tables/timedependent_dataset_final_only_valid_level_allfeat.csv", sep=";")
df_ds = pd.read_csv(data_path + "SPON_data/20181208_update/parsed_data_final.csv", sep=",", low_memory=False, parse_dates=[1])

In [4]:
df_ds.columns

Index(['id', 'date', 'table', 'customer', 'feature', 'domain', 'version',
       'score', 'value', 'level', 'meaning', 'stoerung', 'kredit_jahr',
       'kredit_nutzung', 'kredit_historie', 'allgemein', 'anschrift',
       'all_columns_text', 'activeLoanAmount', 'admonition', 'cashing',
       'employment', 'executionProceedings', 'familyStatus', 'housing',
       'insolvencyProceedings', 'mailOrder', 'migrationBackground',
       'monthlyIncome', 'numberOfActiveLoans', 'numberOfChildren',
       'numberOfCreditCards', 'numberOfGiroAccounts',
       'numberOfMobileContracts', 'numberOfPaidOffLoans', 'paidOffLoanAmount',
       'plz', 'relocation', 'schufaDate', 'sex', 'yearOfBirth', 'foreigner',
       'basisscore'],
      dtype='object')

In [5]:
# Duplicate Check:
# Assuming only one current score per person
tmp = df_ds[(df_ds["table"]=="table2")&(df_ds["domain"]=="Versandhandel")&(df_ds["version"]==3)]
tmp[tmp["id"].duplicated(keep=False)].shape

(0, 43)

In [6]:
df_ds.id.unique().shape

(2625,)

## Questionaire Features

In [7]:
# plz agggregations
def convert_plz(x):
    if str(x) == "nan":
        return x
    else:
        x = str(int(x))
        if len(x) == 4:
            x = "0" + x
        return x

df_ds["plz"] = df_ds["plz"].apply(convert_plz)

In [8]:
def aggregate_plz(x):
    if str(x) == "nan":
        return x
    else:
        return x[0]

df_ds["plz_one"] = df_ds["plz"].apply(aggregate_plz)

In [9]:
def generate_east_west(x):
    if str(x) == "nan":
        return x
    else:
        if x in ["0", "1"]:
            return "east"
        else:
            return "west"
    
df_ds["east_west"] = df_ds["plz_one"].apply(generate_east_west)

In [10]:
def valid_timestamp(x):
    if str(x)=="nan":
        return x
    else:
        if x[:4] <= str(2018):
            return pd.Timestamp(x)
        else:
            return np.nan

df_ds["date"] = df_ds["date"].apply(valid_timestamp)

In [11]:
# age at point of getting the schufa-information
def get_age(row):
    if str(row["yearOfBirth"])=="nan":
        return np.nan
    else:
        return row["date"].year - row["yearOfBirth"]
    
df_ds["age"] = df_ds.apply(get_age, axis=1)

In [12]:
# classes of age according to zensus 2011, bpb

def get_ageclass(row):
    if np.isnan(row):
        return row
    elif row < 15:
        return "<15"
    elif row >= 15 and row < 25:
        return "15-24"
    elif row >= 25 and row < 30:
        return "25-29"
    elif row >= 30 and row < 40:
        return "30-39"
    elif row >= 40 and row < 50:
        return "40-49"
    elif row >= 50 and row < 65:
        return "50-64"
    else:
        return ">=65"
    
df_ds["age_class"] = df_ds["age"].apply(get_ageclass)

In [13]:
gen_feature_map = { "0_17":1, "18_34":2, "35_50":3, "50_over":4 }

def get_generation(x):
    if np.isnan(x):
        return x
    # Generation Z (iGeneration)
    elif x < 18:
        return gen_feature_map["0_17"]
    # Generation Y (Millennials)
    elif x >= 18 and x < 35:
        return gen_feature_map["18_34"]
    # Generation X
    elif x >= 35 and x < 50:
        return gen_feature_map["35_50"]
    # Baby Boomers Generation
    elif x >= 50:
        return gen_feature_map["50_over"]
        
df_ds["age_gen"] = df_ds["age"].apply(get_generation)

### Remove unplausible datapoints

In [14]:
# Remove all schufa-informations before year 2000 and after year 2018
df_ds = df_ds[df_ds["date"].dt.year >= 2000]
df_ds = df_ds[df_ds["date"].dt.year <= 2018]

In [15]:
df_ds["date"].dt.year.value_counts()

2018    51167
2017    11082
2016     2110
2015     1129
2014      869
2013      665
2012      642
2011      637
2010      488
2009      388
2008      271
2007      270
2006      208
2005      153
2004      125
2003       83
2002       63
2001       53
2000       49
Name: date, dtype: int64

## Score Features
### Remove unplausible datapoints

In [16]:
df_ds["level"].value_counts()

A    13571
B    10575
D     5110
C     4872
E     4628
G     4609
F     4599
H     2338
I     1032
M      964
L      624
K      602
N      282
P      209
O      198
S        3
V        3
U        2
R        2
T        1
Name: level, dtype: int64

In [17]:
only_valid = np.all(
    [df_ds["level"]!='V', 
     df_ds["level"]!='U',
     df_ds["level"]!='S',
     df_ds["level"]!='T',
     df_ds["level"]!='R'],
    axis=0)
df_ds = df_ds[only_valid]

In [18]:
# Remove observations with negative features
only_positive = np.all(
    [df_ds["level"]!='N', 
     df_ds["level"]!='O',
     df_ds["level"]!='P'],
    axis=0)
df_ds = df_ds[only_positive]

In [19]:
df_ds["level"].value_counts()

A    13571
B    10575
D     5110
C     4872
E     4628
G     4609
F     4599
H     2338
I     1032
M      964
L      624
K      602
Name: level, dtype: int64

In [20]:
# Cases after cleaning
df_ds.id.unique().shape

(2597,)

In [21]:
df_ds.to_csv(data_path + "datasets_v2/20190703_dataset.csv", index=False, sep=";")

## SCHUFA Auskunft Features
### Analyse SCHUFA Features and find further or more distinct features

In [22]:
table2 = df_ds[df_ds["table"]=="table2"]

In [23]:
# Count the requests per person
amount_requests = df_ds[df_ds["table"]=="table1"].groupby("id")["table"].count()
amount_requests.name = "request"

# Count the text information feature per person
amount_feature = df_ds[df_ds["table"]=="text"].groupby("id")["feature"].value_counts().unstack()

# Join both feature sets with our dataset
table2 = table2.join(amount_requests, on="id")
table2 = table2.join(amount_feature, on="id")

In [24]:
texts = df_ds[df_ds["table"]=="text"]

In [25]:
texts["feature"].value_counts()

giro                          3260
credit_card                   2198
solvency                      2017
conditions                    1290
telecom                       1053
identity                       856
unsecured_loan                 504
secured_loan                   431
business_relation              259
banking_connection             209
global_credit                  134
rent_request                    38
address_update                  23
insufficient_assets             21
undisclosed_assets              16
seizure_protection_account      14
line_of_credit                  12
mail_order                      11
debt_collection                 11
business_giro                    8
business_credit_card             7
debt_relief                      6
lease_purchase                   4
settlement_account               1
permanent_account                1
rent_signed                      1
Name: feature, dtype: int64

In [26]:
table2.columns

Index(['id', 'date', 'table', 'customer', 'feature', 'domain', 'version',
       'score', 'value', 'level', 'meaning', 'stoerung', 'kredit_jahr',
       'kredit_nutzung', 'kredit_historie', 'allgemein', 'anschrift',
       'all_columns_text', 'activeLoanAmount', 'admonition', 'cashing',
       'employment', 'executionProceedings', 'familyStatus', 'housing',
       'insolvencyProceedings', 'mailOrder', 'migrationBackground',
       'monthlyIncome', 'numberOfActiveLoans', 'numberOfChildren',
       'numberOfCreditCards', 'numberOfGiroAccounts',
       'numberOfMobileContracts', 'numberOfPaidOffLoans', 'paidOffLoanAmount',
       'plz', 'relocation', 'schufaDate', 'sex', 'yearOfBirth', 'foreigner',
       'basisscore', 'plz_one', 'east_west', 'age', 'age_class', 'age_gen',
       'request', 'address_update', 'banking_connection',
       'business_credit_card', 'business_giro', 'business_relation',
       'conditions', 'credit_card', 'debt_collection', 'debt_relief', 'giro',
       'global

In [27]:
table2[(table2.domain=="Banken")&(table2.version==3.)].iloc[:, -27:].count().sort_values(ascending=False)

giro                          1223
request                        981
credit_card                    978
solvency                       733
telecom                        550
identity                       423
conditions                     277
unsecured_loan                 208
secured_loan                   176
business_relation              123
banking_connection             111
global_credit                   85
rent_request                    22
address_update                  10
mail_order                       6
line_of_credit                   6
business_credit_card             6
business_giro                    5
debt_collection                  4
seizure_protection_account       4
debt_relief                      3
lease_purchase                   3
undisclosed_assets               2
permanent_account                1
insufficient_assets              1
settlement_account               0
rent_signed                      0
dtype: int64

Features to be removed:
- rent_signed -> too few observations
- settlement_account -> too few observations & negative feature
- insurance_account -> too few observations
- permanent_account -> too few observations
- undisclosed_assets -> negative feature
- lease_purchase -> too few observations
- debt_relief -> too few observations & negative feature
- seizure_protection_account -> negative feature
- debt_collection -> negative feature
- business_giro -> too few observations
- business_credit_card -> too few observations
- line_of_credit -> too few observations
- mail_order -> too few observations
- address_update -> too few observations
- rent_request -> too few observations
- phone_data -> too few observations
- insufficient_assets -> negative feature

(The negative features seem to be "soft" negative features, here we should test their relevance)
=> Negative features are going to be included for now


In [28]:
table2.columns[-27:]

Index(['request', 'address_update', 'banking_connection',
       'business_credit_card', 'business_giro', 'business_relation',
       'conditions', 'credit_card', 'debt_collection', 'debt_relief', 'giro',
       'global_credit', 'identity', 'insufficient_assets', 'lease_purchase',
       'line_of_credit', 'mail_order', 'permanent_account', 'rent_request',
       'rent_signed', 'secured_loan', 'seizure_protection_account',
       'settlement_account', 'solvency', 'telecom', 'undisclosed_assets',
       'unsecured_loan'],
      dtype='object')

In [29]:
table2.iloc[:, -27:] = table2.iloc[:, -27:].fillna(0)

In [30]:
# We want do aggregation because we don't know the information the variables encode
# Hence we just remove people with attributes greater 0, thus controling for this information in the data
"""table2["giro_aggr"] = table2[["business_giro", "giro"]].sum(axis=1)
table2["credit_card_aggr"] = table2[["business_credit_card", "credit_card"]].sum(axis=1)
table2["credit_aggr"] = table2[["line_of_credit", "global_credit"]].sum(axis=1)"""

too_few = ["rent_signed", "settlement_account", "permanent_account",
           "undisclosed_assets", "lease_purchase", "debt_relief", "seizure_protection_account",
           "debt_collection", "business_giro", "business_credit_card", "line_of_credit",
           "mail_order", "address_update", "rent_request", "insufficient_assets"]
            # insurance_account, phone_data

table2_outliers = table2.copy()

for feat in too_few:
    table2 = table2[table2[feat]==0]

In [31]:
table2[too_few].sum()

rent_signed                   0.0
settlement_account            0.0
permanent_account             0.0
undisclosed_assets            0.0
lease_purchase                0.0
debt_relief                   0.0
seizure_protection_account    0.0
debt_collection               0.0
business_giro                 0.0
business_credit_card          0.0
line_of_credit                0.0
mail_order                    0.0
address_update                0.0
rent_request                  0.0
insufficient_assets           0.0
dtype: float64

In [32]:
table2.drop(too_few, axis=1, inplace=True)
table2.columns

Index(['id', 'date', 'table', 'customer', 'feature', 'domain', 'version',
       'score', 'value', 'level', 'meaning', 'stoerung', 'kredit_jahr',
       'kredit_nutzung', 'kredit_historie', 'allgemein', 'anschrift',
       'all_columns_text', 'activeLoanAmount', 'admonition', 'cashing',
       'employment', 'executionProceedings', 'familyStatus', 'housing',
       'insolvencyProceedings', 'mailOrder', 'migrationBackground',
       'monthlyIncome', 'numberOfActiveLoans', 'numberOfChildren',
       'numberOfCreditCards', 'numberOfGiroAccounts',
       'numberOfMobileContracts', 'numberOfPaidOffLoans', 'paidOffLoanAmount',
       'plz', 'relocation', 'schufaDate', 'sex', 'yearOfBirth', 'foreigner',
       'basisscore', 'plz_one', 'east_west', 'age', 'age_class', 'age_gen',
       'request', 'banking_connection', 'business_relation', 'conditions',
       'credit_card', 'giro', 'global_credit', 'identity', 'secured_loan',
       'solvency', 'telecom', 'unsecured_loan'],
      dtype='object'

In [33]:
table2.shape, table2_outliers.shape, table2.id.unique().shape, table2_outliers.id.unique().shape

((46451, 60), (48491, 75), (2136,), (2240,))

### create new count features
We want to create a feature for each branche that counts amount of text-requests and amount of score-requests

In [34]:
# score-requests
#amount_requests = df_ds[df_ds["table"]=="table1"].groupby("id")["table"].count()
#amount_requests.name = "request"

amount_branche_scores = df_ds[df_ds.table=="table1"].groupby("id")["domain"].value_counts().unstack()

amount_branche_scores.columns = amount_branche_scores.columns + "_req"

amount_branche_scores = amount_branche_scores.fillna(0)

table2 = table2.join(amount_branche_scores, on="id")

In [35]:
# On the example of solvency we can see that its not trivial to map the request to a domain..
# (we would need to create a map from each customer to a schufa-domain/branche)
# => we will drop this kind of feature because of time resources
texts[texts.feature=="solvency"][["feature", "domain"]].head()

,feature,domain
61126,solvency,NaN
61130,solvency,NaN
61136,solvency,NaN
61171,solvency,NaN
61173,solvency,NaN


In [36]:
table2.columns

Index(['id', 'date', 'table', 'customer', 'feature', 'domain', 'version',
       'score', 'value', 'level', 'meaning', 'stoerung', 'kredit_jahr',
       'kredit_nutzung', 'kredit_historie', 'allgemein', 'anschrift',
       'all_columns_text', 'activeLoanAmount', 'admonition', 'cashing',
       'employment', 'executionProceedings', 'familyStatus', 'housing',
       'insolvencyProceedings', 'mailOrder', 'migrationBackground',
       'monthlyIncome', 'numberOfActiveLoans', 'numberOfChildren',
       'numberOfCreditCards', 'numberOfGiroAccounts',
       'numberOfMobileContracts', 'numberOfPaidOffLoans', 'paidOffLoanAmount',
       'plz', 'relocation', 'schufaDate', 'sex', 'yearOfBirth', 'foreigner',
       'basisscore', 'plz_one', 'east_west', 'age', 'age_class', 'age_gen',
       'request', 'banking_connection', 'business_relation', 'conditions',
       'credit_card', 'giro', 'global_credit', 'identity', 'secured_loan',
       'solvency', 'telecom', 'unsecured_loan', 'Banken_req',
       

### create features about the time since first contact to finance products
we want the time (in days) since a person had first contact to finance products such as credit-cards, giro accounts, credit-loan. To do that we will take the date of the text-request and count the days till the date of the schufa-auskunft.

In [37]:
texts["schufaDate"] = texts["schufaDate"].apply(lambda x: pd.Timestamp(x))

/home/phil/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [38]:
ids = texts.id.unique()

In [39]:
init_col = np.zeros(len(ids))
init_col[:] = np.nan
dict_df = {"credit_days":init_col, "credit_card_days":init_col, "giro_days":init_col}
temp_df = pd.DataFrame.from_dict(dict_df)
temp_df.index = ids
temp_df.loc[ids[0]]["giro_days"]

nan

In [40]:
texts.feature.value_counts()

giro                          3260
credit_card                   2198
solvency                      2017
conditions                    1290
telecom                       1053
identity                       856
unsecured_loan                 504
secured_loan                   431
business_relation              259
banking_connection             209
global_credit                  134
rent_request                    38
address_update                  23
insufficient_assets             21
undisclosed_assets              16
seizure_protection_account      14
line_of_credit                  12
mail_order                      11
debt_collection                 11
business_giro                    8
business_credit_card             7
debt_relief                      6
lease_purchase                   4
settlement_account               1
permanent_account                1
rent_signed                      1
Name: feature, dtype: int64

In [41]:
texts.groupby("id").get_group(ids[0]).sort_values("date").date

61105   2004-04-29
61106   2016-01-25
61107   2017-09-13
61108   2017-09-18
61109   2017-09-26
61110   2017-09-27
61111   2017-09-27
61112   2018-04-01
Name: date, dtype: datetime64[ns]

In [42]:
giro_d = []
credit_c_d = []
credit_d = []

def days_finance(df):
    has_giro = False
    has_credit = False
    has_credit_card = False
    for idx, row in df.sort_values("date").iterrows():
        if not has_giro and (row["feature"] == "giro" or row["feature"] == "business_giro"):
            has_giro = True
            giro_d.append((row["schufaDate"] - row["date"]).days)
        elif not has_credit_card and (row["feature"] == "credit_card" or row["feature"] == "business_credit_card"):
            has_credit_card = True
            credit_c_d.append((row["schufaDate"] - row["date"]).days)
        elif not has_credit and (row["feature"] == "global_credit" or row["feature"] == "line_of_credit" or row["feature"] == "permanent_account" or row["feature"] == "secured_loan" or row["feature"] == "unsecured_loan"):
            has_credit = True
            credit_d.append((row["schufaDate"] - row["date"]).days)
        else:
            continue
    if not has_giro:
        giro_d.append(np.nan)
    if not has_credit:
        credit_d.append(np.nan)
    if not has_credit_card:
        credit_c_d.append(np.nan)            
    return None
        
texts.groupby("id").apply(days_finance);

In [43]:
temp_df["credit_card_days"] = credit_c_d[1:]
temp_df["credit_days"] = credit_d[1:]
temp_df["giro_days"] = giro_d[1:]

In [44]:
texts[texts.id == ids[0]]["schufaDate"] - texts[texts.id == ids[0]]["date"] # .days to retrieve integer

61105   5142 days
61106    854 days
61107    257 days
61108    252 days
61109    244 days
61110    243 days
61111    243 days
61112     57 days
dtype: timedelta64[ns]

In [45]:
temp_df = temp_df.fillna(0)

In [46]:
table2 = table2.join(temp_df, on="id")

In [47]:
table2[(table2.version==1)&(table2.domain=="Banken")][["level", "credit_card_days", "credit_days", "giro_days"]]

,level,credit_card_days,credit_days,giro_days
9,A,854.0,0.0,5142.0
59,G,4943.0,2636.0,5041.0
93,A,NaN,NaN,NaN
119,F,1385.0,0.0,2466.0
148,A,1851.0,0.0,0.0
174,A,0.0,0.0,5081.0
205,C,116.0,0.0,3649.0
255,L,NaN,NaN,NaN
312,B,575.0,0.0,878.0
342,NaN,0.0,2399.0,1809.0


#### count text-data that is at most one year old

In [48]:
init_col = np.zeros(len(ids))
init_col[:] = np.nan
dict_df = {"finance_year":init_col}
temp_df = pd.DataFrame.from_dict(dict_df)
temp_df.index = ids
temp_df.loc[ids[0]]["finance_year"]

nan

In [49]:
tmp = texts.groupby("id").get_group(ids[0])
tmp = tmp[(tmp.feature=="giro")|(tmp.feature=="credit_card")]
((tmp.schufaDate - tmp.date).apply(lambda x: x.days) <= 365).sum()

3

In [50]:
year_finance = []

def last_year_finance(df):
    
    finance_feats = df[(df.feature=="giro")|
                       (df.feature=="business_giro")|
                       (df.feature=="credit_card")|
                       (df.feature=="business_credit_card")|
                       (df.feature=="global_credit")|
                       (df.feature=="line_of_credit")|
                       (df.feature=="permanent_account")|
                       (df.feature=="secured_loan")|
                       (df.feature=="unsecured_loan")]
    
    if not finance_feats.empty:
        diff_days = finance_feats.schufaDate - finance_feats.date
        amount_this_year = (diff_days.apply(lambda x: x.days) <= 365).sum()
    else:
        amount_this_year = 0.0
        
    year_finance.append(amount_this_year)
    
    return None
        
texts.groupby("id").apply(last_year_finance);

In [51]:
temp_df["finance_year"] = year_finance[1:]

In [52]:
table2 = table2.join(temp_df, on="id")

#### preprocess & replace nans with zeros for covariates

In [54]:
table2.columns[-26:]

Index(['request', 'banking_connection', 'business_relation', 'conditions',
       'credit_card', 'giro', 'global_credit', 'identity', 'secured_loan',
       'solvency', 'telecom', 'unsecured_loan', 'Banken_req',
       'Freiberufler_req', 'Genossenschaftsbanken_req', 'Handel_req',
       'Hypothekengeschäft_req', 'Kleingewerbetreibende_req', 'Sparkassen_req',
       'Sparkassen/Genossenschaftsbanken_req',
       'Telekommunikationsunternehmen_req', 'Versandhandel_req', 'credit_days',
       'credit_card_days', 'giro_days', 'finance_year'],
      dtype='object')

In [55]:
table2.iloc[:, -26:] = table2.iloc[:, -26:].fillna(0)

In [56]:
table2.id.unique().shape

(2136,)

In [57]:
table2.columns

Index(['id', 'date', 'table', 'customer', 'feature', 'domain', 'version',
       'score', 'value', 'level', 'meaning', 'stoerung', 'kredit_jahr',
       'kredit_nutzung', 'kredit_historie', 'allgemein', 'anschrift',
       'all_columns_text', 'activeLoanAmount', 'admonition', 'cashing',
       'employment', 'executionProceedings', 'familyStatus', 'housing',
       'insolvencyProceedings', 'mailOrder', 'migrationBackground',
       'monthlyIncome', 'numberOfActiveLoans', 'numberOfChildren',
       'numberOfCreditCards', 'numberOfGiroAccounts',
       'numberOfMobileContracts', 'numberOfPaidOffLoans', 'paidOffLoanAmount',
       'plz', 'relocation', 'schufaDate', 'sex', 'yearOfBirth', 'foreigner',
       'basisscore', 'plz_one', 'east_west', 'age', 'age_class', 'age_gen',
       'request', 'banking_connection', 'business_relation', 'conditions',
       'credit_card', 'giro', 'global_credit', 'identity', 'secured_loan',
       'solvency', 'telecom', 'unsecured_loan', 'Banken_req',
       

In [60]:
table2.to_csv(data_path + "datasets_v2/20190703_dataset_table2_analysis.csv", index=False, sep=";")